# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [25]:
# all import statements needed for the project, for example:

import os

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
from bs4 import BeautifulSoup
import re
from datetime import datetime
from typing import List
!pip install geopandas
import geopandas as gpd
!pip install fiona shapely pyproj
import math
!pip install pandas pyarrow

In [17]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = r"C:\Users\sanch\TOOLS for Analytics\Project 1\taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = r"C:\Users\sanch\TOOLS for Analytics\Project 1\weather_data"

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [19]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Download Yellow Taxi and HVFHV Data

In [27]:
def get_taxi_html() -> str:
    """Fetch the HTML content of the taxi data page."""
    response = requests.get(TAXI_URL)
    response.raise_for_status()
    html = response.content
    return html

### Load Taxi Zones

In [39]:
def load_taxi_zones(shapefile):
    """
    Load the Taxi Zones shapefile and return a GeoDataFrame.

    Parameters:
    shapefile (str): The path to the Taxi Zones shapefile.

    Returns:
    geopandas.GeoDataFrame: A GeoDataFrame containing the Taxi Zones data.
    """
    try:
        taxi_zones = gpd.read_file(shapefile)
        print(f"Taxi Zones data loaded successfully. Total zones: {len(taxi_zones)}")
        return taxi_zones
    except Exception as e:
        print(f"Error loading Taxi Zones shapefile: {e}")
        return None

In [49]:

# Load the shapefile
taxi_zones = load_taxi_zones(TAXI_ZONES_SHAPEFILE)

# Inspect the loaded data
if taxi_zones is not None:
    print(taxi_zones.head())  # Display

Taxi Zones data loaded successfully. Total zones: 263
   OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0         1    0.116357    0.000782           Newark Airport           1   
1         2    0.433470    0.004866              Jamaica Bay           2   
2         3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3         4    0.043567    0.000112            Alphabet City           4   
4         5    0.092146    0.000498            Arden Heights           5   

         borough                                           geometry  
0            EWR  POLYGON ((933100.918 192536.086, 933091.011 19...  
1         Queens  MULTIPOLYGON (((1033269.244 172126.008, 103343...  
2          Bronx  POLYGON ((1026308.77 256767.698, 1026495.593 2...  
3      Manhattan  POLYGON ((992073.467 203714.076, 992068.667 20...  
4  Staten Island  POLYGON ((935843.31 144283.336, 936046.565 144...  


In [69]:
unique_location_ids = taxi_zones['LocationID'].unique()

# Display the unique Location_ID values
print(unique_location_ids)
print(f"Total unique Location_IDs: {len(unique_location_ids)}")

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 106 107 108 109 110 111
 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129
 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147
 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165
 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183
 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201
 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219
 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237
 238 239 240 241 242 243 244 245 246 247 248 249 25

In [61]:
def lookup_coords_for_taxi_zone_id(zone_loc_id, loaded_taxi_zones):

    """
    Look up the latitude and longitude for a given Taxi Zone location ID.

    Parameters:
    - zone_loc_id (int): The Taxi Zone Location ID.
    - loaded_taxi_zones (GeoDataFrame): The GeoDataFrame containing taxi zones.

    Returns:
    - tuple: (latitude, longitude) of the zone's centroid, or None if not found.
    """
    try:
        # Ensure the GeoDataFrame is in a geographic CRS (EPSG:4326)
        loaded_taxi_zones = loaded_taxi_zones.to_crs(epsg=4326)
        
        # Find the row corresponding to the specified location ID
        zone = loaded_taxi_zones.loc[loaded_taxi_zones['LocationID'] == zone_loc_id]
        
        if zone.empty:
            print(f"location ID {zone_loc_id} not found.")
            return None
        
        # Calculate centroid and extract coordinates
        lon = zone.geometry.centroid.x.values[0]
        lat = zone.geometry.centroid.y.values[0]
        
        return lat, lon
    
    except Exception as e:
        print(f"Error occurred while looking up coordinates: {e}")
        return None

In [71]:
taxi_zones = load_taxi_zones(TAXI_ZONES_SHAPEFILE) # Same as above 
 
coords_list = []

# Loop through LocationID from 1 to 263
for zone_id in range(1, 264):
    coords = lookup_coords_for_taxi_zone_id(zone_id, taxi_zones)
    if coords:
        coords_list.append({'LocationID': zone_id, 'Latitude': coords[0], 'Longitude': coords[1]})

# Convert the list of results into a DataFrame
coords_df = pd.DataFrame(coords_list)

# Display the DataFrame
print(coords_df)

# Optionally, save the results to a CSV file
coords_df.to_csv("taxi_zone_coordinates.csv", index=False)

Taxi Zones data loaded successfully. Total zones: 263


C:\Users\sanch\AppData\Local\Temp\ipykernel_15932\3578696659.py:25: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lon = zone.geometry.centroid.x.values[0]
C:\Users\sanch\AppData\Local\Temp\ipykernel_15932\3578696659.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lat = zone.geometry.centroid.y.values[0]


location ID 57 not found.
location ID 104 not found.
location ID 105 not found.
     LocationID   Latitude  Longitude
0             1  40.691831 -74.174000
1             2  40.616745 -73.831299
2             3  40.864474 -73.847422
3             4  40.723752 -73.976968
4             5  40.552659 -74.188484
..          ...        ...        ...
255         259  40.897932 -73.852215
256         260  40.744235 -73.906306
257         261  40.709139 -74.013023
258         262  40.775932 -73.946510
259         263  40.778766 -73.951010

[260 rows x 3 columns]


### Calculate Sample Size

In [77]:

def calculate_sample_size(population, confidence_level=0.95, margin_of_error=0.05, proportion=0.5):
    """
    Calculate the sample size based on Cochran's formula.
    
    Parameters:
    - population (int): Total population size (i.e., total data points for each month)
    - confidence_level (float): Confidence level (default is 0.95 for 95% confidence)
    - margin_of_error (float): Margin of error (default is 0.05 for 5% margin of error)
    - proportion (float): Estimated proportion of the population (default is 0.5 for maximum sample size)
    
    Returns:
    - int: Required sample size
    """
    # Z-scores for common confidence levels
    z_scores = {
        0.90: 1.645,
        0.95: 1.96,
        0.99: 2.576
    }
    
    # Get the Z-score for the desired confidence level
    Z = z_scores.get(confidence_level, 1.96)  # Default to 95% confidence if not found
    
    # Cochran's sample size formula (n0)
    n0 = (Z ** 2 * proportion * (1 - proportion)) / (margin_of_error ** 2)
    
    # Adjust for finite population if necessary
    if population < 1000:  # If population is small, apply finite population correction
        n = n0 / (1 + (n0 - 1) / population)
    else:
        n = n0
    
    return math.ceil(n)  # Round up to ensure the sample size is sufficient



In [88]:

### Can Delete later: TESTING for SAMPLING only 2020.1 Yellow Taxi

month_file = r"C:\Users\sanch\TOOLS for Analytics\Project 1\taxi_data\yellow_tripdata_2020-01.parquet"
month_data = pd.read_parquet(month_file)

# Get the population size (total records for the month)
population_size = len(month_data)

# Calculate the sample size
sample_size = calculate_sample_size(population_size, confidence_level=0.95, margin_of_error=0.05)
print(f"population size:{population_size}")
print(f"Sample size for this month: {sample_size}")

population size:6405008
Sample size for this month: 385


### Common Functions: Downloading the Yellow Taxi Parquet links

In [5]:

TAXI_URL: str = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# Date range for filtering
START_DATE = datetime(2020, 1, 1)
END_DATE = datetime(2024, 8, 30)

def get_taxi_html() -> str:
    """Fetch the HTML content of the taxi data page."""
    response = requests.get(TAXI_URL)
    response.raise_for_status()
    html = response.content
    return html

def find_taxi_parquet_links() -> List[str]:
    """Find links to Yellow Taxi and HVFHV Parquet files within the date range."""
    html = get_taxi_html()
    soup = bs4.BeautifulSoup(html, "html.parser")
    
    # Find all <a> tags with relevant titles
    yellow_a_tags = soup.find_all("a", attrs={"title": "Yellow Taxi Trip Records"})
 
    
    # Combine all links
    all_a_tags = yellow_a_tags
    
    # Extract href attributes and filter based on .parquet
    parquet_links = [a["href"].strip() for a in all_a_tags if ".parquet" in (a.get("href") or "")]
    return filter_links_by_date(parquet_links)

def filter_links_by_date(links: List[str]) -> List[str]:
    """Filter Parquet file links by date, retaining only those within the specified range."""
    filtered_links = []
    date_pattern = re.compile(r"_(\d{4})-(\d{2})\.parquet")
    
    for link in links:
        match = date_pattern.search(link)
        if match:
            year, month = int(match.group(1)), int(match.group(2))
            file_date = datetime(year, month, 1)
            if START_DATE <= file_date <= END_DATE:
                filtered_links.append(link)
    
    return filtered_links

def download_files(links: List[str], folder_name: str) -> None:
    """Download files from a list of links and save them to the specified folder."""
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
    
    for link in links:
        file_name = link.split("/")[-1]
        file_path = os.path.join(folder_name, file_name)
        print(f"Downloading {file_name} from {link}...")
        if os.path.exists(file_path):
            print(f"File {file_name} already exists. Skipping download.")
            continue  # Skip to the next file if it already exists
        
        print(f"Downloading {file_name} from {link}...")
        
        # Request with headers to mimic a browser
        response = requests.get(link, headers=headers)
        response.raise_for_status()  # Check if download was successful
        
        with open(file_path, "wb") as file:
            file.write(response.content)
        print(f"Downloaded {file_name}")

# Find and download filtered links
filtered_links = find_taxi_parquet_links()
download_files(filtered_links, "yellow_taxi")

File yellow_tripdata_2024-01.parquet already exists. Skipping download.
File yellow_tripdata_2024-02.parquet already exists. Skipping download.
File yellow_tripdata_2024-03.parquet already exists. Skipping download.
File yellow_tripdata_2024-04.parquet already exists. Skipping download.
File yellow_tripdata_2024-05.parquet already exists. Skipping download.
File yellow_tripdata_2024-06.parquet already exists. Skipping download.
File yellow_tripdata_2024-07.parquet already exists. Skipping download.
File yellow_tripdata_2024-08.parquet already exists. Skipping download.
File yellow_tripdata_2023-01.parquet already exists. Skipping download.
File yellow_tripdata_2023-02.parquet already exists. Skipping download.
File yellow_tripdata_2023-03.parquet already exists. Skipping download.
File yellow_tripdata_2023-04.parquet already exists. Skipping download.
File yellow_tripdata_2023-05.parquet already exists. Skipping download.
File yellow_tripdata_2023-06.parquet already exists. Skipping do

### COMMON FUNCTION: Dowloading the HVFHV Parquet links

In [7]:
def get_all_urls_from_tlc_page() -> str:
    """Fetch the HTML content of the taxi data page."""
    response = requests.get(TAXI_URL)
    response.raise_for_status()  # Raise an error for bad responses
    html = response.content
    return html
TAXI_URL: str = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

# Date range for filtering
START_DATE = datetime(2020, 1, 1)
END_DATE = datetime(2024, 8, 30)

def get_taxi_html() -> str:
    """Fetch the HTML content of the taxi data page."""
    response = requests.get(TAXI_URL)
    response.raise_for_status()
    html = response.content
    return html

def find_taxi_parquet_links() -> List[str]:
    """Find links to Yellow Taxi and HVFHV Parquet files within the date range."""
    html = get_taxi_html()
    soup = bs4.BeautifulSoup(html, "html.parser")
    
    # Find all <a> tags with relevant titles
    hvfhv_a_tags = soup.find_all("a", href=re.compile(r"fhvhv.*\.parquet", re.IGNORECASE))
 
    
    # Combine all links
    all_a_tags = hvfhv_a_tags
    
    # Extract href attributes and filter based on .parquet
    parquet_links = [a["href"].strip() for a in all_a_tags if ".parquet" in (a.get("href") or "")]
    return filter_links_by_date(parquet_links)

def filter_links_by_date(links: List[str]) -> List[str]:
    """Filter Parquet file links by date, retaining only those within the specified range."""
    filtered_links = []
    date_pattern = re.compile(r"_(\d{4})-(\d{2})\.parquet")
    
    for link in links:
        match = date_pattern.search(link)
        if match:
            year, month = int(match.group(1)), int(match.group(2))
            file_date = datetime(year, month, 1)
            if START_DATE <= file_date <= END_DATE:
                filtered_links.append(link)
    
    return filtered_links

def download_files(links: List[str], folder_name: str) -> None:
    """Download files from a list of links and save them to the specified folder."""
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
    
    for link in links:
        file_name = link.split("/")[-1]
        file_path = os.path.join(folder_name, file_name)
        print(f"Downloading {file_name} from {link}...")

        if os.path.exists(file_path):
            print(f"File {file_name} already exists. Skipping download.")
            continue  # Skip to the next file if it already exists
        
        print(f"Downloading {file_name} from {link}...")
        
        # Request with headers to mimic a browser
        response = requests.get(link, headers=headers)
        response.raise_for_status()  # Check if download was successful
        
        with open(file_path, "wb") as file:
            file.write(response.content)
        print(f"Downloaded {file_name}")

# Find and download filtered links
filtered_links = find_taxi_parquet_links()
download_files(filtered_links, "fhvhv_raw")



File fhvhv_tripdata_2024-01.parquet already exists. Skipping download.
File fhvhv_tripdata_2024-02.parquet already exists. Skipping download.
File fhvhv_tripdata_2024-03.parquet already exists. Skipping download.
File fhvhv_tripdata_2024-04.parquet already exists. Skipping download.
File fhvhv_tripdata_2024-05.parquet already exists. Skipping download.
File fhvhv_tripdata_2024-06.parquet already exists. Skipping download.
File fhvhv_tripdata_2024-07.parquet already exists. Skipping download.
File fhvhv_tripdata_2024-08.parquet already exists. Skipping download.
File fhvhv_tripdata_2023-01.parquet already exists. Skipping download.
File fhvhv_tripdata_2023-02.parquet already exists. Skipping download.
File fhvhv_tripdata_2023-03.parquet already exists. Skipping download.
File fhvhv_tripdata_2023-04.parquet already exists. Skipping download.
File fhvhv_tripdata_2023-05.parquet already exists. Skipping download.
File fhvhv_tripdata_2023-06.parquet already exists. Skipping download.
File f

### FILTERING ONLY UBER DATA from FHVHV parquets

In [40]:
source_folder = r"C:\Users\sanch\TOOLS for Analytics\Project 1\fhvhv_raw"
output_folder = r"C:\Users\sanch\TOOLS for Analytics\Project 1\processed_uber_data"

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created output folder: {output_folder}")
else:
    print(f"The folder '{output_folder}' already exists.")

# List all Parquet files in the source folder
files = [os.path.join(source_folder, f) for f in os.listdir(source_folder) if f.endswith('.parquet')]

# Process each file
for file_path in files:
    # Extract the part of the filename after "fhvhv" to create the new file name
    base_name = os.path.basename(file_path)  # Extracts the filename with extension
    new_file_name = f"uber_{base_name.split('fhvhv')[-1]}"  # Create new filename with "uber_" prefix
    output_file_path = os.path.join(output_folder, new_file_name)  # Full output path

    # Check if the output file already exists
    if os.path.exists(output_file_path):
        print(f"Output file {output_file_path} already exists. Skipping processing for {file_path}.")
        continue

    # Read the Parquet file into a DataFrame
    df = pd.read_parquet(file_path)

    # Filter the rows where 'hvfhs_license_num' is 'HV0003'
    filtered_df = df[df['hvfhs_license_num'] == 'HV0003']

    # Save the filtered DataFrame to a new Parquet file
    filtered_df.to_parquet(output_file_path)

    print(f"Filtered data from {file_path} saved to {output_file_path}")

The folder 'C:\Users\sanch\TOOLS for Analytics\Project 1\processed_uber_data' already exists.
Output file C:\Users\sanch\TOOLS for Analytics\Project 1\processed_uber_data\uber__tripdata_2020-01.parquet already exists. Skipping processing for C:\Users\sanch\TOOLS for Analytics\Project 1\fhvhv_raw\fhvhv_tripdata_2020-01.parquet.
Output file C:\Users\sanch\TOOLS for Analytics\Project 1\processed_uber_data\uber__tripdata_2020-02.parquet already exists. Skipping processing for C:\Users\sanch\TOOLS for Analytics\Project 1\fhvhv_raw\fhvhv_tripdata_2020-02.parquet.
Output file C:\Users\sanch\TOOLS for Analytics\Project 1\processed_uber_data\uber__tripdata_2020-03.parquet already exists. Skipping processing for C:\Users\sanch\TOOLS for Analytics\Project 1\fhvhv_raw\fhvhv_tripdata_2020-03.parquet.
Output file C:\Users\sanch\TOOLS for Analytics\Project 1\processed_uber_data\uber__tripdata_2020-04.parquet already exists. Skipping processing for C:\Users\sanch\TOOLS for Analytics\Project 1\fhvhv_ra

### Process Taxi Data

In [ ]:
def get_and_clean_taxi_month(url):    #### QUESTION?: IS it after SAMPLING each month's data 
    raise NotImplementedError()

In [ ]:
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month(parquet_url)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

In [ ]:
def get_taxi_data():
    all_urls = get_all_urls_from_taxi_page(TLC_URL)
    all_parquet_urls = find_taxi_parquet_urls(all_urls)
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    return taxi_data

In [ ]:
taxi_data = get_taxi_data()

In [ ]:
taxi_data.head()

In [ ]:
taxi_data.info()

In [ ]:
taxi_data.describe()

### Processing Uber Data

In [ ]:
def get_and_clean_uber_month(url):
    raise NotImplementedError()

In [ ]:
def get_and_clean_uber_data(parquet_urls):
    all_uber_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_uber_month(parquet_url)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_uber_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    uber_data = pd.contact(all_uber_dataframes)
    return uber_data

In [ ]:
def load_and_clean_uber_data():
    raise NotImplementedError()

In [ ]:
def get_uber_data():
    all_urls = get_all_urls_from_tlc_page(TLC_URL)
    all_parquet_urls = find_parquet_urls(all_urls)
    taxi_data = get_and_clean_uber_data(all_parquet_urls)
    return taxi_data

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

In [ ]:
uber_data.info()

In [ ]:
uber_data.describe()

### Processing Weather Data

In [ ]:
def get_all_weather_csvs(directory):
    raise NotImplementedError()

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    raise NotImplementedError()

In [ ]:
def clean_month_weather_data_daily(csv_file):
    raise NotImplementedError()

In [ ]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
hourly_weather_data.head()

In [ ]:
hourly_weather_data.info()

In [ ]:
hourly_weather_data.describe()

In [ ]:
daily_weather_data.head()

In [ ]:
daily_weather_data.info()

In [ ]:
daily_weather_data.describe()

## Part 2: Storing Cleaned Data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [ ]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [ ]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_1)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_1, con=engine)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)